# Synthesize Data 

## 0 - Setup

### 0.1 - Packages 

In [1]:
import pandas as pd
from numpy import repeat

from ydata.metadata import Metadata
from ydata.synthesizers.regular import RegularSynthesizer
from ydata.connectors import LocalConnector
from ydata.connectors.filetype import FileType
from ydata.dataset import Dataset

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass memory=None, verbose=False as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


## 1 - Load Data

In [2]:
# Read the data (Only if you want to train a new network)
data = pd.read_csv('data_processed.csv')
#Filter the data by the unusual observations
data = data.query("Unusual == 1")

In [3]:
data = data.drop(['Time', 'CellName'], axis=1)

## 2 - Synthesization 

### 2.1 - Train the Synthesizer 

In [4]:
#create the dataset. This allows scale and integration with YData's platform SDK's
dataset = Dataset(data)
#Calculate the metadata
m = Metadata(dataset)

[########################################] | 100% Completed |  0.7s


In [5]:
print(m)

Metadata Summary 
 
Dataset type: TABULAR
Dataset attributes: 
Number of columns: 9
% of duplicate rows: 0
Target column: 

Column detail: 
        Column    Data type Variable type
0   PRBUsageUL    numerical         float
1   PRBUsageDL    numerical         float
2   meanThr_DL    numerical         float
3   meanThr_UL    numerical         float
4    maxThr_DL    numerical         float
5    maxThr_UL    numerical         float
6    meanUE_UL    numerical         float
7  maxUE_UL+DL  categorical           int
8      Unusual  categorical           int

Warnings: 
                   Warning                                                                 Columns
0     WarningType.SKEWNESS  [PRBUsageUL, PRBUsageDL, meanThr_DL, meanThr_UL, maxThr_DL, maxThr_UL]
1  WarningType.CARDINALITY                                                           [maxUE_UL+DL]



In [6]:
# Initialize and Load an alerady trained Synthesizer.
network_synth = RegularSynthesizer()

network_synth.fit(dataset, metadata=m)
network_synth.save('trained_model.pkl')

#Load a pre-saved model
#network_synth = network_synth.load('trained_model.pkl')

INFO: 2022-07-26 07:44:48,930 [SYNTHESIZER] - Number columns considered for synth: 9
INFO: 2022-07-26 07:45:00,978 [SYNTHESIZER] - Starting the synthetic data modeling process over 1x1 blocks.
INFO: 2022-07-26 07:45:00,983 [SYNTHESIZER] - Preprocess segment
INFO: 2022-07-26 07:45:03,788 [SYNTHESIZER] - Synthesizer init.
INFO: 2022-07-26 07:45:03,790 [SYNTHESIZER] - Processing the data prior fitting the synthesizer.


### 2.2 - Sample Data 

In [7]:
# Create the samples and flag them.
synth_sample = network_synth.sample(5000).to_pandas()

# Flag the created samples and assign columns 
synth_sample.index = repeat(999999, len(synth_sample))
synth_sample.columns = data.columns

INFO: 2022-07-26 07:45:05,842 [SYNTHESIZER] - Start generating model samples.


### 3 - Store Data 

In [8]:
synth_sample.to_csv('data_sampled.csv', index=False)